<a href="https://colab.research.google.com/github/abar-1/nbaPredictor/blob/main/NBA_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nba_api scikit-learn pandas numpy xgboost matplotlib seaborn --quiet

print("Starting NBA Stat Predictor...")
players = ["Domantas Sabonis"]
PLAYER_NAME = "Domantas Sabonis"
# --- Imports ---
import pandas as pd
import numpy as np
import os
import json
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playergamelog, leaguedashteamstats, playerprofilev2, scoreboardv2
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import warnings
import time
from datetime import datetime, timedelta

warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Constants ---
STATS_TO_PREDICT = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'MIN', 'FG3M', 'PRA']

# True Shooting constant (free throw factor from NBA)
TS_FT_FACTOR = 0.44

BASE_PLAYER_FEATURES = [
    'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
    'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
    'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
    'TS_PCT', 'GAME_SCORE', 'PRA', 'USG_ESTIMATE'
]

ROLLING_FEATURES_TO_CALC = ['MIN', 'FGA', 'PTS', 'REB', 'AST', 'PRA', 'GAME_SCORE', 'TS_PCT']

OPPONENT_DEFENSE_FEATURES = [
    'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
    'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
    'DEF_RATING', 'PACE', 'OPP_EFG_PCT', 'OPP_OREB_PCT', 'OPP_TOV_PCT'
]

CONTEXT_FEATURES = ['IS_HOME', 'DAYS_REST', 'BACK_TO_BACK']

# --- Static Data ---
TEAMS_LIST = teams.get_teams()
TEAMS_DF = pd.DataFrame(TEAMS_LIST)
PLAYERS_LIST = players.get_players()
PLAYERS_DF = pd.DataFrame(PLAYERS_LIST)

# --- Caching ---
CACHE_DIR = "nba_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

def cache_path(filename):
    return os.path.join(CACHE_DIR, filename)

def save_to_cache(data, filename):
    try:
        with open(cache_path(filename), "w") as f:
            json.dump(data, f)
    except Exception as e:
        print(f"Warning: Failed to save cache {filename}: {e}")

def load_from_cache(filename):
    try:
        with open(cache_path(filename), "r") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return None

# --- Helper Functions ---
def get_player_id(player_name):
    player = PLAYERS_DF[PLAYERS_DF['full_name'].str.lower() == player_name.lower()]
    return player.iloc[0]['id'] if not player.empty else None

def get_team_id(team_abbr):
    team = TEAMS_DF[TEAMS_DF['abbreviation'] == team_abbr]
    return team.iloc[0]['id'] if not team.empty else None

def get_team_abbr(team_id):
    team = TEAMS_DF[TEAMS_DF['id'] == team_id]
    return team.iloc[0]['abbreviation'] if not team.empty else None

def get_training_seasons():
    """Gets current season and 4 previous seasons."""
    today = datetime.now()
    current_season_start_year = today.year if today.month >= 10 else today.year - 1

    seasons = [
        f"{current_season_start_year - i}-{str(current_season_start_year - i + 1)[-2:]}"
        for i in range(4, -1, -1)
    ]

    print(f"Training seasons: {seasons}")
    return seasons

# --- API Helpers ---
def fetch_with_retry_all_dfs(endpoint_class, attempts=3, timeout=15, **kwargs):
    for attempt in range(attempts):
        try:
            time.sleep(0.6)
            api_call = endpoint_class(timeout=timeout, **kwargs)
            return api_call.get_data_frames()
        except Exception as e:
            if 'timeout' in str(e).lower() and attempt < attempts - 1:
                time.sleep(1)
            else:
                raise e
    raise Exception(f"Failed to fetch from {endpoint_class.__name__}")

def fetch_with_retry(endpoint_class, attempts=3, timeout=15, **kwargs):
    dfs = fetch_with_retry_all_dfs(endpoint_class, attempts, timeout, **kwargs)
    return dfs[0] if dfs and len(dfs) > 0 else pd.DataFrame()

# --- Data Fetching ---
def fetch_player_gamelogs(player_id, seasons):
    all_games_df = []
    print(f"Fetching gamelogs for player {player_id}...")

    for season in seasons:
        cache_file = f"player_{player_id}_{season}_gamelog.json"
        cached_data = load_from_cache(cache_file)

        df = pd.DataFrame()
        if cached_data:
            df = pd.DataFrame(cached_data)
        else:
            try:
                df = fetch_with_retry(playergamelog.PlayerGameLog, player_id=player_id, season=season)
                if not df.empty:
                    save_to_cache(df.to_dict(orient="records"), cache_file)
            except Exception as e:
                print(f"Note: Could not fetch {season}")
                df = pd.DataFrame()

        if not df.empty:
            df['SEASON_ID'] = season
            all_games_df.append(df)

    if not all_games_df:
        return pd.DataFrame()

    full_df = pd.concat(all_games_df, ignore_index=True)
    full_df['GAME_DATE'] = pd.to_datetime(full_df['GAME_DATE'])
    full_df = full_df.sort_values(by='GAME_DATE').reset_index(drop=True)

    # --- FIX: Filter out DNP games (where MIN is 0) ---
    # This prevents 0-stat games from poisoning the season averages.
    full_df = full_df[full_df['MIN'] > 0].reset_index(drop=True)
    # --- END FIX ---

    print(f"Found {len(full_df)} games")
    return full_df

def fetch_all_team_defensive_stats(seasons):
    print(f"Fetching team defensive stats...")
    season_stats_dict = {}

    for season in seasons:
        cache_file = f"team_stats_merged_{season}.json"
        cached_data = load_from_cache(cache_file)

        if cached_data:
            season_stats_dict[season] = pd.DataFrame(cached_data)
            continue

        try:
            df_opp = fetch_with_retry(
                leaguedashteamstats.LeagueDashTeamStats,
                season=season, measure_type_detailed_defense='Opponent', per_mode_detailed='PerGame'
            )
            df_adv = fetch_with_retry(
                leaguedashteamstats.LeagueDashTeamStats,
                season=season, measure_type_detailed_defense='Advanced', per_mode_detailed='PerGame'
            )
            df_four = fetch_with_retry(
                leaguedashteamstats.LeagueDashTeamStats,
                season=season, measure_type_detailed_defense='Four Factors', per_mode_detailed='PerGame'
            )

            opp_rename = {
                'TEAM_ID': 'TEAM_ID', 'OPP_PTS': 'PTS', 'OPP_FGM': 'FGM', 'OPP_FGA': 'FGA',
                'OPP_FG_PCT': 'FG_PCT', 'OPP_FG3M': 'FG3M', 'OPP_FG3A': 'FG3A', 'OPP_FG3_PCT': 'FG3_PCT',
                'OPP_FTM': 'FTM', 'OPP_FTA': 'FTA', 'OPP_FT_PCT': 'FT_PCT',
                'OPP_OREB': 'OREB', 'OPP_DREB': 'DREB', 'OPP_REB': 'REB',
                'OPP_AST': 'AST', 'OPP_TOV': 'TOV', 'OPP_STL': 'STL', 'OPP_BLK': 'BLK'
            }
            df_opp = df_opp[opp_rename.keys()].rename(columns=opp_rename)
            df_adv = df_adv[['TEAM_ID', 'DEF_RATING', 'PACE']]
            df_four = df_four[['TEAM_ID', 'OPP_EFG_PCT', 'OPP_OREB_PCT', 'OPP_TOV_PCT']]

            df_merged = pd.merge(df_opp, df_adv, on='TEAM_ID')
            df_merged = pd.merge(df_merged, df_four, on='TEAM_ID')
            df_final = pd.merge(df_merged, TEAMS_DF[['id', 'abbreviation']], left_on='TEAM_ID', right_on='id')
            df_final.rename(columns={'abbreviation': 'TEAM_ABBREVIATION'}, inplace=True)
            df_final.drop(columns=['id'], inplace=True, errors='ignore')

            save_to_cache(df_final.to_dict(orient="records"), cache_file)
            season_stats_dict[season] = df_final

        except Exception as e:
            print(f"Error fetching {season}: {e}")

    return season_stats_dict

# --- Feature Engineering ---
def calculate_advanced_stats(df):
    """Calculate advanced stats with safety checks."""
    df = df.copy()

    # True Shooting Percentage
    df['TS_PCT'] = df['PTS'] / (2 * (df['FGA'] + TS_FT_FACTOR * df['FTA']) + 1e-6)

    # Game Score (John Hollinger's formula)
    df['GAME_SCORE'] = (
        df['PTS'] + 0.4 * df['FGM'] - 0.7 * df['FGA'] - 0.4 * (df['FTA'] - df['FTM']) +
        0.7 * df['OREB'] + 0.3 * df['DREB'] + df['STL'] + 0.7 * df['AST'] +
        0.7 * df['BLK'] - 0.4 * df['PF'] - df['TOV']
    )

    # Points + Rebounds + Assists
    df['PRA'] = df['PTS'] + df['REB'] + df['AST']

    # Usage estimate (simplified)
    df['USG_ESTIMATE'] = (df['FGA'] + 0.44 * df['FTA'] + df['TOV']) / (df['MIN'] + 1e-6)

    return df

def create_features(player_df, team_stats_dict):
    if player_df.empty:
        return pd.DataFrame()

    df = player_df.copy()
    df = calculate_advanced_stats(df)

    # Target variables (next game stats)
    for stat in STATS_TO_PREDICT:
        df[f'TARGET_{stat}'] = df[stat].shift(-1)

    # Context features
    df['IS_HOME'] = (~df['MATCHUP'].str.contains('@')).astype(int)
    df['OPP_ABBR'] = df['MATCHUP'].str.split(' ').str[-1]
    df['DAYS_REST'] = df['GAME_DATE'].diff().dt.days
    df['BACK_TO_BACK'] = (df['DAYS_REST'] == 1).astype(int)

    # Season averages and rolling windows
    grouped = df.groupby('SEASON_ID')

    for stat in BASE_PLAYER_FEATURES:
        # Expanding mean (season average up to this point)
        df[f'{stat}_SEASON_AVG'] = grouped[stat].transform(lambda x: x.shift(1).expanding().mean())
        df[f'{stat}_SEASON_STD'] = grouped[stat].transform(lambda x: x.shift(1).expanding().std())

        # Rolling averages for hot/cold streaks
        if stat in ROLLING_FEATURES_TO_CALC:
            df[f'{stat}_ROLL_3_AVG'] = grouped[stat].transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
            df[f'{stat}_ROLL_7_AVG'] = grouped[stat].transform(lambda x: x.shift(1).rolling(7, min_periods=3).mean())
            df[f'{stat}_ROLL_3_STD'] = grouped[stat].transform(lambda x: x.shift(1).rolling(3, min_periods=1).std())

    # Merge opponent defensive stats
    all_season_dfs = []
    fallback_season = sorted(team_stats_dict.keys())[-1] if team_stats_dict else None

    for season_id, group in df.groupby('SEASON_ID'):
        team_stats_df = team_stats_dict.get(season_id, team_stats_dict.get(fallback_season))

        if team_stats_df is None:
            continue

        rename_map = {col: f'OPP_{col}' for col in OPPONENT_DEFENSE_FEATURES}
        merged_group = pd.merge(
            group, team_stats_df.rename(columns=rename_map),
            left_on='OPP_ABBR', right_on='TEAM_ABBREVIATION', how='left'
        )
        all_season_dfs.append(merged_group)

    if not all_season_dfs:
        return pd.DataFrame()

    final_df = pd.concat(all_season_dfs, ignore_index=True)

    # Clean up
    target_cols = [f'TARGET_{stat}' for stat in STATS_TO_PREDICT]
    final_df = final_df.dropna(subset=target_cols)

    avg_cols = [f'{stat}_SEASON_AVG' for stat in BASE_PLAYER_FEATURES]
    final_df = final_df.dropna(subset=avg_cols)
    final_df = final_df.fillna(0) # Fill NaNs (rolling std, days_rest)

    return final_df

def create_prediction_features(player_df, team_stats_dict, next_opp_abbr, is_home, days_rest):
    if player_df.empty:
        return None

    last_game = player_df.iloc[-1]
    features = {}

    last_game_season = last_game['SEASON_ID']
    season_games = player_df[player_df['SEASON_ID'] == last_game_season].copy()
    season_games = calculate_advanced_stats(season_games)

    # Rolling averages
    for stat in ROLLING_FEATURES_TO_CALC:
        last_3 = season_games.iloc[-3:][stat]
        last_7 = season_games.iloc[-7:][stat]
        features[f'{stat}_ROLL_3_AVG'] = last_3.mean()
        features[f'{stat}_ROLL_7_AVG'] = last_7.mean()
        features[f'{stat}_ROLL_3_STD'] = last_3.std() if len(last_3) > 1 else 0

    # Season averages and std
    for stat in BASE_PLAYER_FEATURES:
        features[f'{stat}_SEASON_AVG'] = season_games[stat].mean()
        features[f'{stat}_SEASON_STD'] = season_games[stat].std() if len(season_games) > 1 else 0

    # Context
    features['IS_HOME'] = int(is_home)
    features['DAYS_REST'] = days_rest
    features['BACK_TO_BACK'] = int(days_rest == 1)

    # Opponent stats
    latest_season = player_df['SEASON_ID'].iloc[-1]
    opp_stats_df = team_stats_dict.get(latest_season, team_stats_dict[sorted(team_stats_dict.keys())[-1]])

    if opp_stats_df is None:
        return None

    opp_stats = opp_stats_df[opp_stats_df['TEAM_ABBREVIATION'] == next_opp_abbr]

    if opp_stats.empty:
        print(f"Warning: No stats for {next_opp_abbr}, using league average")
        opp_stats_avg = opp_stats_df[OPPONENT_DEFENSE_FEATURES].mean()
        for col in OPPONENT_DEFENSE_FEATURES:
            features[f'OPP_{col}'] = opp_stats_avg.get(col, 0)
    else:
        for col in OPPONENT_DEFENSE_FEATURES:
            features[f'OPP_{col}'] = opp_stats.iloc[0].get(col, 0)

    return pd.DataFrame([features]).fillna(0)

# --- Enhanced Model Training ---
def get_feature_subset(stat_name):
    """Enhanced feature subsets with new advanced stats."""
    features = ['IS_HOME', 'DAYS_REST', 'BACK_TO_BACK']

    if stat_name != 'MIN':
        features.append('MIN_SEASON_AVG')

    player_features = {
        'PTS': ['PTS_SEASON_AVG', 'PTS_ROLL_3_AVG', 'PTS_ROLL_7_AVG', 'PTS_ROLL_3_STD',
                'TS_PCT_SEASON_AVG', 'TS_PCT_ROLL_3_AVG', 'GAME_SCORE_SEASON_AVG',
                'FGA_ROLL_3_AVG', 'USG_ESTIMATE_SEASON_AVG'],
        'REB': ['REB_SEASON_AVG', 'REB_ROLL_3_AVG', 'REB_SEASON_STD',
                'OREB_SEASON_AVG', 'DREB_SEASON_AVG'],
        'AST': ['AST_SEASON_AVG', 'AST_ROLL_3_AVG', 'AST_SEASON_STD',
                'TOV_SEASON_AVG', 'GAME_SCORE_SEASON_AVG', 'USG_ESTIMATE_SEASON_AVG'],
        'STL': ['STL_SEASON_AVG', 'PF_SEASON_AVG'],
        'BLK': ['BLK_SEASON_AVG', 'PF_SEASON_AVG'],
        'MIN': ['MIN_SEASON_AVG', 'MIN_ROLL_3_AVG', 'MIN_SEASON_STD',
                'PF_SEASON_AVG', 'GAME_SCORE_ROLL_3_AVG'],
        'FG3M': ['FG3M_SEASON_AVG', 'FG3A_SEASON_AVG', 'FG3M_ROLL_3_AVG',
                 'FG3_PCT_SEASON_AVG'],
        'PRA': ['PRA_SEASON_AVG', 'PRA_ROLL_3_AVG', 'PRA_ROLL_7_AVG',
                'GAME_SCORE_SEASON_AVG', 'GAME_SCORE_ROLL_3_AVG', 'TS_PCT_SEASON_AVG']
    }

    opp_features = {
        'PTS': ['OPP_DEF_RATING', 'OPP_PACE', 'OPP_EFG_PCT'],
        'REB': ['OPP_REB', 'OPP_OREB_PCT'],
        'AST': ['OPP_AST', 'OPP_TOV_PCT', 'OPP_PACE'],
        'STL': ['OPP_STL', 'OPP_TOV_PCT'],
        'BLK': ['OPP_BLK', 'OPP_FGA'],
        'MIN': ['OPP_DEF_RATING', 'OPP_PACE'],
        'FG3M': ['OPP_DEF_RATING', 'OPP_FG3A', 'OPP_FG3_PCT'],
        'PRA': ['OPP_DEF_RATING', 'OPP_PACE', 'OPP_EFG_PCT']
    }

    features.extend(player_features.get(stat_name, []))
    features.extend(opp_features.get(stat_name, []))

    return list(dict.fromkeys(features)) # Return unique features in order

def calculate_baseline_metrics(y_train, y_test):
    """Calculate simple baseline predictions for comparison."""
    baselines = {}

    # Baseline 1: Season average
    season_avg = y_train.mean()
    mae_season = mean_absolute_error(y_test, [season_avg] * len(y_test))
    baselines['Season Avg'] = {'MAE': mae_season, 'prediction': season_avg}

    return baselines

def train_model_with_validation(X_all, y, stat_name):
    """Enhanced training with time-series validation and baselines."""
    print(f"\n{'='*50}")
    print(f"Training Model for {stat_name}")
    print(f"{'='*50}")

    feature_subset = get_feature_subset(stat_name)
    valid_features = [f for f in feature_subset if f in X_all.columns]
    X = X_all[valid_features]

    if len(X) < 20:
        print(f"Warning: Only {len(X)} samples. Model may be unreliable.")
        if len(X) < 5:
            return None, None, None

    # Chronological split
    split_idx = int(len(X) * 0.80)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    # Calculate baselines
    baselines = calculate_baseline_metrics(y_train, y_test)

    # Simpler param grid for robustness
    param_grid = {
        'max_depth': [3, 4],
        'learning_rate': [0.05, 0.1],
        'n_estimators': [100, 150],
        'subsample': [0.8],
        'colsample_bytree': [0.8]
    }

    # Use TimeSeriesSplit for proper validation
    n_splits = max(2, min(3, len(X_train) // 10)) # At least 10 samples per split

    model = None
    best_params = {}

    if len(X_train) > (n_splits * 5): # Check for enough data for CV
        tscv = TimeSeriesSplit(n_splits=n_splits)
        xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
        grid_search = GridSearchCV(
            estimator=xgb, param_grid=param_grid, cv=tscv,
            scoring='neg_mean_absolute_error', n_jobs=-1, verbose=0
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        print("Warning: Not enough data for TimeSeriesSplit CV. Fitting simple model.")
        model = XGBRegressor(objective='reg:squarederror', random_state=42, max_depth=3, n_estimators=100)
        model.fit(X_train, y_train)
        best_params = {'max_depth': 3, 'n_estimators': 100} # Default params

    # Test metrics
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    medae = median_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    # Calculate prediction intervals (simple method)
    residuals = y_test - preds
    std_residual = np.std(residuals)

    print(f"Features: {len(valid_features)}")
    print(f"Training samples: {len(X_train)} | Test samples: {len(X_test)}")
    print(f"Best params: {best_params}")
    print(f"\nModel Performance:")
    print(f"  MAE:        {mae:.2f}")
    print(f"  Median AE:  {medae:.2f}")
    print(f"  R²:         {r2:.3f}")
    print(f"  Std Error:  ±{std_residual:.2f}")
    print(f"\nBaseline Comparison:")
    print(f"  Season Avg MAE: {baselines['Season Avg']['MAE']:.2f}")

    # --- FIX: Handle baseline_mae == 0 ---
    if baselines['Season Avg']['MAE'] == 0:
        improvement = 0.0
    else:
        improvement = ((baselines['Season Avg']['MAE'] - mae) / baselines['Season Avg']['MAE']) * 100
    # --- END FIX ---
    print(f"  Improvement: {improvement:+.1f}%")

    # Feature importance
    importance = model.feature_importances_
    top_features = sorted(zip(valid_features, importance), key=lambda x: x[1], reverse=True)[:5]
    print(f"\nTop 5 Features:")
    for feat, imp in top_features:
        print(f"  {feat}: {imp:.3f}")

    # Refit on all data
    final_model = XGBRegressor(objective='reg:squarederror', random_state=42, **best_params)
    final_model.fit(X, y)

    metrics = {
        'mae': mae, 'medae': medae, 'r2': r2, 'std_error': std_residual,
        'baseline_mae': baselines['Season Avg']['MAE'],
        'improvement_pct': improvement
    }

    return final_model, valid_features, metrics

# ============================================================================
# LOGGING & PREDICTION
# ============================================================================

def log_prediction_to_csv(player_name, game_date, opponent, is_home, days_rest, predictions):
    """Saves a prediction record to CSV."""
    log_file = cache_path("prediction_log.csv")

    log_entry = {
        'Prediction_ID': f"{player_name}_{game_date.strftime('%Y%m%d')}",
        'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'Player_Name': player_name,
        'Game_Date': game_date.strftime('%Y-%m-%d'),
        'Opponent': opponent,
        'Matchup': 'HOME' if is_home else 'AWAY',
        'Days_Rest': days_rest,
        'Status': 'PENDING'
    }

    # Add predictions with confidence intervals
    for stat, pred_data in predictions.items():
        if pred_data:
            log_entry[f"Pred_{stat}"] = pred_data['prediction']
            log_entry[f"Pred_{stat}_Lower"] = pred_data['lower_95']
            log_entry[f"Pred_{stat}_Upper"] = pred_data['upper_95']
            log_entry[f"Actual_{stat}"] = None  # Will be filled later

    new_log_df = pd.DataFrame([log_entry])

    try:
        if os.path.exists(log_file):
            log_df = pd.read_csv(log_file)
            # Check if prediction already exists
            existing = log_df[log_df['Prediction_ID'] == log_entry['Prediction_ID']]
            if not existing.empty:
                print(f"⚠️  Prediction for {log_entry['Prediction_ID']} already exists. Skipping log.")
                return
            log_df = pd.concat([log_df, new_log_df], ignore_index=True)
        else:
            log_df = new_log_df

        log_df.to_csv(log_file, index=False)
        print(f"✅ Prediction logged: {log_entry['Prediction_ID']}")

    except Exception as e:
        print(f"Warning: Failed to log prediction: {e}")


def update_prediction_with_actuals(player_name, game_date, actual_stats):
    """Updates a prediction log with actual game results."""
    log_file = cache_path("prediction_log.csv")

    if not os.path.exists(log_file):
        print("❌ No prediction log found.")
        return False

    try:
        log_df = pd.read_csv(log_file)
        prediction_id = f"{player_name}_{game_date.strftime('%Y%m%d')}"

        mask = log_df['Prediction_ID'] == prediction_id

        if not mask.any():
            print(f"❌ No prediction found for {prediction_id}")
            return False

        # Update actual values
        for stat, value in actual_stats.items():
            if f"Actual_{stat}" in log_df.columns:
                log_df.loc[mask, f"Actual_{stat}"] = value

        log_df.loc[mask, 'Status'] = 'COMPLETED'
        log_df.loc[mask, 'Updated_At'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        log_df.to_csv(log_file, index=False)
        print(f"✅ Updated actuals for {prediction_id}")

        # Calculate and display accuracy
        row = log_df[mask].iloc[0]
        print("\n" + "="*60)
        print(f"📊 PREDICTION vs ACTUAL for {player_name}")
        print("="*60)

        for stat in STATS_TO_PREDICT:
            pred_col = f"Pred_{stat}"
            actual_col = f"Actual_{stat}"

            if pred_col in row and actual_col in row and pd.notna(row[pred_col]) and pd.notna(row[actual_col]):
                pred = row[pred_col]
                actual = row[actual_col]
                error = actual - pred
                pct_error = (error / (actual + 1e-6)) * 100

                # Check if within confidence interval
                lower = row.get(f"Pred_{stat}_Lower", pred)
                upper = row.get(f"Pred_{stat}_Upper", pred)
                in_ci = "✅" if lower <= actual <= upper else "❌"

                print(f"{stat:5} | Pred: {pred:5.1f} | Actual: {actual:5.1f} | " +
                      f"Error: {error:+5.1f} ({pct_error:+.0f}%) | In 95% CI: {in_ci}")

        print("="*60)
        return True

    except Exception as e:
        print(f"❌ Error updating actuals: {e}")
        return False


def fetch_actual_stats_for_game(player_name, game_date):
    """
    Fetches actual game stats for a player on a specific date.
    Returns dict of actual stats or None if not found.
    """
    player_id = get_player_id(player_name)
    if not player_id:
        return None

    # Determine which season the game_date falls in
    game_year = game_date.year
    if game_date.month >= 10:  # Oct-Dec
        season = f"{game_year}-{str(game_year + 1)[-2:]}"
    else:  # Jan-Sep
        season = f"{game_year - 1}-{str(game_year)[-2:]}"

    print(f"Fetching actual stats for {player_name} on {game_date.strftime('%Y-%m-%d')}...")

    try:
        gamelog_df = fetch_player_gamelogs(player_id, [season])

        if gamelog_df.empty:
            print("No gamelogs found.")
            return None

        # Find the game on the specific date
        # Need to convert GAME_DATE to date() for comparison
        game_row = gamelog_df[gamelog_df['GAME_DATE'].dt.date == game_date.date()]

        if game_row.empty:
            print(f"No game found on {game_date.strftime('%Y-%m-%d')}")
            return None

        game = game_row.iloc[0]

        # Calculate PRA
        pra = game['PTS'] + game['REB'] + game['AST']

        actual_stats = {
            'PTS': game['PTS'],
            'REB': game['REB'],
            'AST': game['AST'],
            'STL': game['STL'],
            'BLK': game['BLK'],
            'MIN': game['MIN'],
            'FG3M': game['FG3M'],
            'PRA': pra
        }

        print(f"✅ Found actual stats for game on {game_date.strftime('%Y-%m-%d')}")
        return actual_stats

    except Exception as e:
        print(f"Error fetching actual stats: {e}")
        return None


def auto_update_pending_predictions(player_name=None):
    """
    Automatically checks for pending predictions and updates them with actuals.
    If player_name is provided, only updates that player's predictions.
    """
    log_file = cache_path("prediction_log.csv")

    if not os.path.exists(log_file):
        print("No prediction log found.")
        return

    try:
        log_df = pd.read_csv(log_file)

        # Filter for pending predictions
        pending = log_df[log_df['Status'] == 'PENDING'].copy()

        if player_name:
            pending = pending[pending['Player_Name'].str.lower() == player_name.lower()]

        if pending.empty:
            print("No pending predictions to update.")
            return

        print(f"\n{'='*60}")
        print(f"Found {len(pending)} pending prediction(s) to update")
        print(f"{'='*60}\n")

        for idx, row in pending.iterrows():
            pred_player = row['Player_Name']
            pred_date = pd.to_datetime(row['Game_Date'])

            # Only update if game date has passed
            if pred_date.date() > datetime.now().date():
                print(f"⏳ {pred_player} vs {row['Opponent']} on {pred_date.strftime('%Y-%m-%d')} - Game hasn't occurred yet")
                continue

            print(f"🔄 Updating: {pred_player} vs {row['Opponent']} on {pred_date.strftime('%Y-%m-%d')}")

            actual_stats = fetch_actual_stats_for_game(pred_player, pred_date)

            if actual_stats:
                update_prediction_with_actuals(pred_player, pred_date, actual_stats)
            else:
                print(f"⚠️  Could not fetch actuals for {pred_player} on {pred_date.strftime('%Y-%m-%d')}\n")

    except Exception as e:
        print(f"Error in auto-update: {e}")


def generate_accuracy_report(player_name=None, min_predictions=5):
    """
    Generates a comprehensive accuracy report from completed predictions.
    """
    log_file = cache_path("prediction_log.csv")

    if not os.path.exists(log_file):
        print("❌ No prediction log found. Make predictions first!")
        return

    try:
        log_df = pd.read_csv(log_file)

        # Filter for completed predictions
        completed = log_df[log_df['Status'] == 'COMPLETED'].copy()

        if player_name:
            completed = completed[completed['Player_Name'].str.lower() == player_name.lower()]

        if len(completed) < min_predictions:
            print(f"⚠️  Only {len(completed)} completed prediction(s). Need at least {min_predictions} for meaningful analysis.")
            return

        print("\n" + "="*70)
        print(f"📈 ACCURACY REPORT - {len(completed)} Completed Predictions")
        if player_name:
            print(f"Player: {player_name}")
        print("="*70)

        # Calculate metrics for each stat
        results = []

        for stat in STATS_TO_PREDICT:
            pred_col = f"Pred_{stat}"
            actual_col = f"Actual_{stat}"
            lower_col = f"Pred_{stat}_Lower"
            upper_col = f"Pred_{stat}_Upper"

            if pred_col not in completed.columns or actual_col not in completed.columns:
                continue

            # Remove rows with missing data
            valid = completed[[pred_col, actual_col]].dropna()

            if len(valid) == 0:
                continue

            predictions = valid[pred_col].values
            actuals = valid[actual_col].values

            # Calculate metrics
            mae = mean_absolute_error(actuals, predictions)
            medae = median_absolute_error(actuals, predictions)

            # Mean Absolute Percentage Error
            mape = np.mean(np.abs((actuals - predictions) / (actuals + 1e-6))) * 100

            # Confidence interval coverage (if available)
            ci_coverage = None
            if lower_col in completed.columns and upper_col in completed.columns:
                ci_valid = completed[[actual_col, lower_col, upper_col]].dropna()
                if len(ci_valid) > 0:
                    in_ci = ((ci_valid[actual_col] >= ci_valid[lower_col]) &
                             (ci_valid[actual_col] <= ci_valid[upper_col]))
                    ci_coverage = in_ci.sum() / len(ci_valid) * 100

            # Calculate baseline (season average)
            season_avg = actuals.mean()
            baseline_mae = mean_absolute_error(actuals, [season_avg] * len(actuals))

            # --- FIX: Handle baseline_mae == 0 ---
            if baseline_mae == 0:
                improvement = 0.0
            else:
                improvement = ((baseline_mae - mae) / baseline_mae) * 100
            # --- END FIX ---

            results.append({
                'Stat': stat,
                'Count': len(valid),
                'MAE': mae,
                'MedAE': medae,
                'MAPE': mape,
                'Baseline_MAE': baseline_mae,
                'Improvement': improvement,
                'CI_Coverage': ci_coverage
            })

        # Display results table
        results_df = pd.DataFrame(results)

        print(f"\n{'Stat':<6} {'Count':<7} {'MAE':<7} {'MedAE':<7} {'MAPE':<8} " +
              f"{'Base MAE':<10} {'Improv':<9} {'CI Cov':<8}")
        print("-" * 70)

        for _, row in results_df.iterrows():
            ci_str = f"{row['CI_Coverage']:.1f}%" if pd.notna(row['CI_Coverage']) else "N/A"
            print(f"{row['Stat']:<6} {row['Count']:<7} {row['MAE']:<7.2f} {row['MedAE']:<7.2f} " +
                  f"{row['MAPE']:<7.1f}% {row['Baseline_MAE']:<10.2f} " +
                  f"{row['Improvement']:+7.1f}% {ci_str:<8}")

        # Overall summary
        print("\n" + "="*70)
        print("📊 SUMMARY")
        print("="*70)
        avg_improvement = results_df['Improvement'].mean()
        avg_ci_coverage = results_df['CI_Coverage'].dropna().mean() if not results_df['CI_Coverage'].dropna().empty else None

        print(f"Average Improvement over Baseline: {avg_improvement:+.1f}%")
        if avg_ci_coverage:
            print(f"Average 95% CI Coverage: {avg_ci_coverage:.1f}% (Target: 95%)")
            if abs(avg_ci_coverage - 95) < 5:
                print("✅ Confidence intervals well-calibrated!")
            elif avg_ci_coverage < 90:
                print("⚠️  Confidence intervals may be too narrow (underconfident)")
            else:
                print("⚠️  Confidence intervals may be too wide (overconfident)")

        # Best and worst performing stats
        best_stat = results_df.loc[results_df['Improvement'].idxmax()]
        worst_stat = results_df.loc[results_df['Improvement'].idxmin()]

        print(f"\n🏆 Best Prediction: {best_stat['Stat']} ({best_stat['Improvement']:+.1f}% improvement)")
        print(f"📉 Needs Work: {worst_stat['Stat']} ({worst_stat['Improvement']:+.1f}% improvement)")

        # Recent performance trend
        if len(completed) >= 10:
            recent_5 = completed.tail(5)
            older_5 = completed.iloc[-10:-5] if len(completed) >= 10 else completed.iloc[:-5]

            print(f"\n📈 RECENT TREND (Last 5 vs Previous 5 predictions)")
            for stat in STATS_TO_PREDICT:
                pred_col = f"Pred_{stat}"
                actual_col = f"Actual_{stat}"

                if pred_col in recent_5.columns and actual_col in recent_5.columns:
                    recent_valid = recent_5[[pred_col, actual_col]].dropna()
                    older_valid = older_5[[pred_col, actual_col]].dropna()

                    if len(recent_valid) > 0 and len(older_valid) > 0:
                        recent_mae = mean_absolute_error(recent_valid[actual_col], recent_valid[pred_col])
                        older_mae = mean_absolute_error(older_valid[actual_col], older_valid[pred_col])

                        trend = "📈" if recent_mae < older_mae else "📉"
                        print(f"{stat}: Recent MAE {recent_mae:.2f} vs Old MAE {older_mae:.2f} {trend}")

        # --- FIX: Corrected the stray text in the print statement ---
        print("="*70 + "\n")
        # --- END FIX ---

    except Exception as e:
        print(f"Error generating report: {e}")
        import traceback
        traceback.print_exc()


# --- Prediction with Confidence Intervals ---
def predict_next_game(player_name, seasons, next_opp_abbr, is_home, days_rest, player_df=None):
    player_id = get_player_id(player_name)
    if player_id is None:
        print(f"Error: Player '{player_name}' not found.")
        return None, None

    if player_df is None:
        player_df = fetch_player_gamelogs(player_id, seasons)
        if player_df.empty:
            return None, None

    team_stats_dict = fetch_all_team_defensive_stats(seasons)
    if not team_stats_dict:
        return None, None

    feature_df = create_features(player_df, team_stats_dict)
    if feature_df.empty:
        return None, None

    # Exclude non-features
    exclude_cols = [col for col in feature_df.columns if
                    col.startswith('TARGET_') or col in BASE_PLAYER_FEATURES or
                    col in ['Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'SEASON_ID',
                            'OPP_ABBR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'Player_ID',
                            'VIDEO_AVAILABLE'] or '_DROP' in col]

    feature_columns = [col for col in feature_df.columns if col not in exclude_cols]
    X_all = feature_df[feature_columns]

    prediction_features_df = create_prediction_features(
        player_df, team_stats_dict, next_opp_abbr, is_home, days_rest
    )

    if prediction_features_df is None:
        return None, None

    try:
        prediction_features_df = prediction_features_df[feature_columns]
    except KeyError as e:
        print(f"Feature mismatch: {e}")
        return None, None

    final_predictions = {}
    all_metrics = {}

    for stat in STATS_TO_PREDICT:
        y = feature_df[f'TARGET_{stat}']

        model, features_used, metrics = train_model_with_validation(X_all, y, stat)

        if model and features_used:
            X_pred = prediction_features_df[features_used]
            prediction = model.predict(X_pred)[0]

            # Add confidence interval
            std_error = metrics['std_error']
            lower_bound = max(0, prediction - 1.96 * std_error)  # 95% CI
            upper_bound = prediction + 1.96 * std_error

            final_predictions[stat] = {
                'prediction': max(0, round(float(prediction), 2)),
                'lower_95': round(float(lower_bound), 2),
                'upper_95': round(float(upper_bound), 2),
                'std_error': round(float(std_error), 2),
                'model_mae': round(metrics.get('mae', 0), 2),
                'model_r2': round(metrics.get('r2', 0), 3),
                'model_improvement': round(metrics.get('improvement_pct', 0), 1)
            }
            all_metrics[stat] = metrics
        else:
            final_predictions[stat] = None

    # Display results
    print("\n" + "="*60)
    print(f"🎯 PREDICTIONS FOR {player_name}")
    print("="*60)
    print(f"Matchup: {('HOME' if is_home else 'AWAY')} vs. {next_opp_abbr} (Rest: {days_rest} days)")
    print("="*60)

    for stat in ['MIN', 'PRA'] + [s for s in STATS_TO_PREDICT if s not in ['MIN', 'PRA']]:
        if stat in final_predictions and final_predictions[stat]:
            pred = final_predictions[stat]
            print(f"{stat:5} | Prediction: {pred['prediction']:5.1f} | " +
                  f"95% CI: [{pred['lower_95']:5.1f}, {pred['upper_95']:5.1f}] | " +
                  f"±{pred['std_error']:.1f}")
    print("="*60)

    return final_predictions, all_metrics


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":

    # --- !! MODIFY THIS !! ---
     # This is now the ONLY input
    # --- !! END MODIFY !! ---

    # 1. Auto-detect seasons
    SEASONS_TO_TRAIN = get_training_seasons()

    # 2. Auto-detect next game
    print(f"\n--- Attempting to auto-detect next game for {PLAYER_NAME} ---")

    NEXT_OPPONENT_ABBR = None
    IS_NEXT_GAME_HOME = None
    DAYS_REST_BEFORE_GAME = None
    player_df_for_rest = None
    next_game_date = None
    found_game_flag = False

    try:
        player_id = get_player_id(PLAYER_NAME)
        if player_id is None:
            raise ValueError(f"Player '{PLAYER_NAME}' not found.")

        player_df_for_rest = fetch_player_gamelogs(player_id, SEASONS_TO_TRAIN)
        if player_df_for_rest.empty:
            raise ValueError(f"No gamelogs found for {PLAYER_NAME}.")

        # --- FIX: Find the last game *actually played* ---
        today_date = pd.to_datetime(datetime.now().date())
        played_games = player_df_for_rest[player_df_for_rest['GAME_DATE'] < today_date]
        if played_games.empty:
            raise ValueError(f"No *past* gamelogs found for {PLAYER_NAME} to calculate rest.")
        last_game_date = played_games.iloc[-1]['GAME_DATE']
        # --- END FIX ---

        print("Fetching next game details (Method 1: PlayerProfile)...")
        profile_dfs = fetch_with_retry_all_dfs(
            playerprofilev2.PlayerProfileV2,
            player_id=player_id
        )

        overview_df = profile_dfs[0] if len(profile_dfs) > 0 else pd.DataFrame()
        next_game_df = profile_dfs[1] if len(profile_dfs) > 1 else pd.DataFrame()

        player_team_id = None
        player_team_abbr = None

        if overview_df.empty:
            print("Warning: PlayerProfile overview is empty. Getting team from last gamelog.")
            last_game_matchup = player_df_for_rest.iloc[-1]['MATCHUP']
            player_team_abbr = last_game_matchup.split(' ')[0]
            player_team_id = get_team_id(player_team_abbr)
            if player_team_id is None:
                raise ValueError(f"Could not determine Team ID for {player_team_abbr}")
        else:
            player_team_id = overview_df.iloc[0]['TEAM_ID']
            player_team_abbr = overview_df.iloc[0]['TEAM_ABBREVIATION']
        print(f"Player's team identified as: {player_team_abbr} (ID: {player_team_id})")

        if not next_game_df.empty:
            print("✅ Found game via PlayerProfile.")
            next_game = next_game_df.iloc[0]
            next_game_date = pd.to_datetime(next_game['GAME_DATE'])

            if player_team_id == next_game['HOME_TEAM_ID']:
                IS_NEXT_GAME_HOME = True
                NEXT_OPPONENT_ABBR = next_game['VISITOR_TEAM_ABBREVIATION']
            else:
                IS_NEXT_GAME_HOME = False
                NEXT_OPPONENT_ABBR = next_game['HOME_TEAM_ABBREVIATION']

            DAYS_REST_BEFORE_GAME = (next_game_date - last_game_date).days
            found_game_flag = True

        else:
            print(f"Warning: PlayerProfile shows no upcoming game. Trying scoreboard...")

            server_today_dt = datetime.now().date()
            server_yesterday_dt = server_today_dt - timedelta(days=1)
            check_dates = [server_yesterday_dt, server_today_dt]

            for check_date in check_dates:
                if found_game_flag: break
                check_date_str = check_date.strftime('%m/%d/%Y')
                print(f"Checking scoreboard for {check_date_str}...")
                game_headers = pd.DataFrame()
                try:
                    game_headers = fetch_with_retry(
                        scoreboardv2.ScoreboardV2,
                        game_date=check_date_str
                    )
                except Exception as e:
                    print(f"No games found on {check_date_str}.")

                if not game_headers.empty:
                    for idx, game in game_headers.iterrows():
                        if str(game['HOME_TEAM_ID']) == str(player_team_id):
                            IS_NEXT_GAME_HOME = True
                            NEXT_OPPONENT_ABBR = get_team_abbr(game['VISITOR_TEAM_ID'])
                            next_game_date = pd.to_datetime(game['GAME_DATE_EST'])
                            found_game_flag = True
                            break
                        elif str(game['VISITOR_TEAM_ID']) == str(player_team_id):
                            IS_NEXT_GAME_HOME = False
                            NEXT_OPPONENT_ABBR = get_team_abbr(game['HOME_TEAM_ID'])
                            next_game_date = pd.to_datetime(game['GAME_DATE_EST'])
                            found_game_flag = True
                            break

                if found_game_flag:
                    DAYS_REST_BEFORE_GAME = (next_game_date - last_game_date).days
                    break

            if not found_game_flag:
                raise ValueError(f"No upcoming game found for {player_team_abbr}.")

        print(f"✅ Auto-detected matchup: {('HOME' if IS_NEXT_GAME_HOME else 'AWAY')} vs. {NEXT_OPPONENT_ABBR}")
        print(f"✅ Auto-detected rest: {DAYS_REST_BEFORE_GAME} days (Last: {last_game_date.date()}, Next: {next_game_date.date()})")

    except Exception as e:
        print(f"❌ FATAL ERROR during auto-detection: {e}")
        print("Cannot proceed without next game details.")

    # 3. Make prediction if we have all the info
    if all([NEXT_OPPONENT_ABBR, IS_NEXT_GAME_HOME is not None, DAYS_REST_BEFORE_GAME is not None,
            player_df_for_rest is not None and not player_df_for_rest.empty]):
        try:
            predictions, metrics = predict_next_game(
                PLAYER_NAME,
                SEASONS_TO_TRAIN,
                NEXT_OPPONENT_ABBR,
                IS_NEXT_GAME_HOME,
                DAYS_REST_BEFORE_GAME,
                player_df=player_df_for_rest
            )

            if predictions and next_game_date:
                # Log prediction
                log_prediction_to_csv(
                    PLAYER_NAME,
                    next_game_date,
                    NEXT_OPPONENT_ABBR,
                    IS_NEXT_GAME_HOME,
                    DAYS_REST_BEFORE_GAME,
                    predictions
                )

                print("\n" + "="*60)
                print("🔧 BACKTESTING TOOLS")
                print("="*60)
                print("To update this prediction with actual results after the game:")
                print(f"  auto_update_pending_predictions('{PLAYER_NAME}')")
                print("\nTo generate accuracy report:")
                print(f"  generate_accuracy_report('{PLAYER_NAME}')")
                print("="*60)

            else:
                print("\n❌ Prediction failed during model training.")

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("\n❌ Skipping prediction - auto-detection failed.")

    # 4. Optionally auto-update any pending predictions
    print("\n" + "="*60)
    print("🔄 Checking for pending predictions to update...")
    print("="*60)
    auto_update_pending_predictions(PLAYER_NAME)

    # 5. Show accuracy report if we have enough data
    print("\n" + "="*60)
    print("📊 Generating Accuracy Report...")
    print("="*60)
    generate_accuracy_report(PLAYER_NAME, min_predictions=1)



Starting NBA Stat Predictor...
Training seasons: ['2021-22', '2022-23', '2023-24', '2024-25', '2025-26']

--- Attempting to auto-detect next game for Domantas Sabonis ---
Fetching gamelogs for player 1627734...
Found 297 games
Fetching next game details (Method 1: PlayerProfile)...
Player's team identified as: SAC (ID: 1610612758)
Checking scoreboard for 10/31/2025...
Checking scoreboard for 11/01/2025...
✅ Auto-detected matchup: AWAY vs. MIL
✅ Auto-detected rest: 3 days (Last: 2025-10-29, Next: 2025-11-01)
Fetching team defensive stats...

Training Model for PTS
Features: 15
Training samples: 232 | Test samples: 59
Best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.8}

Model Performance:
  MAE:        5.22
  Median AE:  4.50
  R²:         -0.119
  Std Error:  ±6.48

Baseline Comparison:
  Season Avg MAE: 4.90
  Improvement: -6.6%

Top 5 Features:
  TS_PCT_SEASON_AVG: 0.103
  USG_ESTIMATE_SEASON_AVG: 0.098
  TS_PCT_ROLL_3_A

In [ ]:
import sys
!{sys.executable} -m pip install nba_api


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.0/287.0 kB 4.4 MB/s eta 0:00:00
